## **eFigure 1. SQL Code snippet for encounter information**

In [0]:
set @TSQL_vent_enc=
'with vents_from_table as (
    select 
        E.ENCNTR_ID,
        PI_GET_CV_DISPLAY(E.LOC_NURSE_UNIT_CD) UNIT,
        PI_GET_CV_DISPLAY(E.MED_SERVICE_CD) MED_SERVICE,
        (SELECT distinct
        MIN(ELH.BEG_EFFECTIVE_DT_TM)
        FROM CUST_UTC.VW_ENCNTR_LOC_HIST  ELH
        WHERE elh.ENCNTR_ID =  e.encntr_id
        AND (
           (elh.MED_SERVICE_CD IN (' + @listOfService + ')
                      )
               OR 
               (elh.LOC_NURSE_UNIT_CD IN (' + @listOfUnit + ')
          )  
          )
        )ICU_START_DT_TM,
        (SELECT trunc(TRUNC(sysdate-P.BIRTH_DT_TM) / 365.25)
         FROM  CUST_UTC.VW_PERSON P 
         WHERE E.PERSON_ID = P.PERSON_ID
         AND P.ACTIVE_IND = 1
            )AGE,
        ce.event_id,
        pi_get_cv_display(ce.event_cd) event,
        ce.result_val, 
        ce.event_end_dt_tm        
        from encounter e, clinical_event  ce      -- for performance reason, using table first, will use view in next step
        where  e.encntr_id = ce.encntr_id         
        and e.encntr_class_cd in (310109, 310110) --ED patient , inpatient
        and e.active_ind = 1 
		and e.disch_dt_tm is null        
        and ce.event_cd = 240019457   /* Ventilator Mode */
        and ce.event_end_dt_tm >= sysdate -2         
        and ce.result_status_cd +0 in (26,35,24) /* AUTH(FINAL), MODIFIED, ALTERED(modified) */
        and ce.record_status_cd +0 = 186
        and ce.view_level +0 = 1
		and ce.valid_until_dt_tm > sysdate
),
vents as (     
    select                           
        ce.person_id,CE.ENCNTR_ID, UNIT, MED_SERVICE,ICU_START_DT_TM,AGE,ce.event_id,pi_get_cv_display(ce.event_cd) event, ce.result_val, ce.event_end_dt_tm        
        from vents_from_table vt, cust_utc.vw_clinical_event ce  --using view
        where 
        vt.event_id  = ce.event_id 
        and ce.event_cd = 240019457             /* Ventilator Mode */
        and ce.event_end_dt_tm >= sysdate -1    -- previous 24 hours        
        and ce.result_status_cd +0 in (26,35,24) /* AUTH(FINAL), MODIFIED, ALTERED(modified) */
        and ce.record_status_cd +0 = 186
        and ce.view_level +0 = 1
		and ce.valid_until_dt_tm > sysdate
),   
VENT_ENCNTR_POPULATION AS (
    select distinct v.encntr_id, unit, med_service,ICU_START_DT_TM, AGE from vents v 
), 
vents_and_excubs as (   -- using union to combine all vents and excubations
    SELECT      -- get excubation in previous 24 hours
        ce.person_id, ce.encntr_id,unit,med_service, ICU_START_DT_TM,AGE, ce.event_id,pi_get_cv_display(ce.event_cd) event, ce.result_val, 
        cdr.result_dt_tm event_end_dt_tm
        from cust_utc.vw_DCP_FORMS_ACTIVITY dfa, cust_utc.vw_clinical_event ce , cust_utc.vw_ce_date_result cdr, vent_encntr_population 
        where  dfa.ENCNTR_ID +0 = vent_encntr_population.encntr_id 
        and ce.event_id = cdr.event_id 
        and DFA.DCP_FORMS_REF_ID = 968046 /* Extubation */                   
        and dfa.FORM_STATUS_CD +0 in (26,35,24) 
        and ce.reference_nbr like (to_char(dfa.DCP_FORMS_ACTIVITY_ID) || ''%'')
        and Ce.VALID_UNTIL_DT_TM >= sysdate
        and ce.CONTRIBUTOR_SYSTEM_CD = 455
        and ce.event_cd = 48921238  /* Extubation Time */                           
        and ce.EVENT_END_DT_TM >= sysdate -1    
        and ce.Event_end_dt_tm <= sysdate
        and ce.result_status_cd +0 in (26,35,24) /* AUTH(FINAL), MODIFIED, ALTERED(modified) */
        and ce.record_status_cd +0 = 186
        and ce.view_level +0 = 1
        and ce.publish_flag +0 = 1          
    union 
    select * from vents  where vents.event_end_dt_tm <  sysdate    -- vents 
),
ordered_vents_and_excubs as (       
    select ve.*, ROW_NUMBER() over (partition by encntr_id order by encntr_id, event_end_dt_tm desc) row_num
    from vents_and_excubs ve
),
vents_no_excubs as (
    select
		person_id, ENCNTR_ID, UNIT, MED_SERVICE,ICU_START_DT_TM, AGE,event_id, event,result_val,event_end_dt_tm 
	from ordered_vents_and_excubs where row_num =1 and event = ''Ventilator Mode''  
),
invasive_no_excubs as (
    select * from vents_no_excubs v where 
    upper(v.result_val) in ( 
		''AC/PRESSURE'', 
		''AC/PRESSUREPCV+ASSIST'',  
		''AC/VOLUME'',    
        ''AC-PRVC/AUTOFLOW'',
		''AC/TCPL'', 
		''BILEVEL/APRV'', 
		''CPAP/PSUPP'',
		''CPAP/BIPAP'', 
		''CPAP/PSV'',
		''HFJV'', 
		''HFOV'', 
		''PC-AC/VG'', 
		''PC-SIMV/VG'', 
		''SIMV/AUTOFLOW'', 
		''SIMV/PRVC'', 
		''SIMV/PRESSURE'', 
		''SIMV/PRESSUREPCV+PSUPP'',
		''SIMV/TCPL'',
		''SIMV/VOLUME'',
		''SERVO-BIVENT/APRV'',        
		''AUTOSET'',
		''AVAP-AUTO'',
		''AVAPS'',
		''IMV/NIVNEONATAL'') or (upper(v.result_val) like ''OTHER%'')
),
non_invasive_vents as (      -- get vents from cust_utc.vw_clinical_event
    select  *  from vents_no_excubs v 
        where upper(v.result_val) not in ( 
		''AC/PRESSURE'', 
		''AC/PRESSUREPCV+ASSIST'',  
		''AC/VOLUME'',    
        ''AC-PRVC/AUTOFLOW'',
		''AC/TCPL'', 
		''BILEVEL/APRV'', 
		''CPAP/PSUPP'',
		''CPAP/BIPAP'', 
		''CPAP/PSV'',
		''HFJV'', 
		''HFOV'', 
		''PC-AC/VG'', 
		''PC-SIMV/VG'', 
		''SIMV/AUTOFLOW'', 
		''SIMV/PRVC'', 
		''SIMV/PRESSURE'', 
		''SIMV/PRESSUREPCV+PSUPP'',
		''SIMV/TCPL'',
		''SIMV/VOLUME'',
		''SERVO-BIVENT/APRV'',        
		''AUTOSET'',
		''AVAP-AUTO'',
		''AVAPS'',
		''IMV/NIVNEONATAL'') or (upper(v.result_val) like ''OTHER%'')

),		
 non_invasive_disc_orders as(
select 
distinct 
nv.person_id, 
nv.ENCNTR_ID,
nv.UNIT,
nv.MED_SERVICE,
nv.ICU_START_DT_TM,
nv.AGE, 
o.order_id,
''Discontinued'',
pi_get_cv_display(o.catalog_cd) result_val,
od.oe_field_dt_tm_value
from cust_utc.vw_orders o,cust_utc.vw_order_detail od, non_invasive_vents nv 
where o.encntr_id = nv.encntr_id 
and o.catalog_cd in 
(97386437,	--BiPAP Ventilation
110905043,	--High Flow Nasal Cannula
48967160,	--High Frequency Jet Ventilation
48967156,	--High Frequency Oscillatory Ventilation
48967151,	--Ventilator Airway Pressure Release Venti
48967134,	--Ventilator CMV - Pressure Control
48967128,	--Ventilator CMV - Volume Control
48967138,	--Ventilator CPAP
48967146,	--Ventilator IMV - Pressure Control
48967142,	--Ventilator IMV - Volume Control
305441644	--Ventilator Non-Invasive IMV
)
and o.order_id = od.order_id
and od.oe_field_id = 10730
and od.oe_field_dt_tm_value >= sysdate -1 
and od.oe_field_dt_tm_value < sysdate 
order by nv.encntr_id,  od.oe_field_dt_tm_value desc
),
non_invasive_and_disc as (  
   select * from non_invasive_vents
   union
   select * from non_invasive_disc_orders  
),
ordered_non_invasive_and_disc as
(     select nivd.*, ROW_NUMBER() over (partition by encntr_id order by encntr_id, event_end_dt_tm desc) row_num
    from non_invasive_and_disc nivd
 ),
non_invasive_no_disc as (
  select * from ordered_non_invasive_and_disc where row_num =1 and event <> ''Discontinued''
),
vents_on_pop as ( 
select * from invasive_no_excubs
union 
select   
		person_id,	ENCNTR_ID,	UNIT,MED_SERVICE,ICU_START_DT_TM,AGE,event_id,event,result_val,	event_end_dt_tm
		from non_invasive_no_disc
)
 SELECT 
DISTINCT ENCNTR_ID,
event_end_dt_tm,
''VENT'' POPULATION,
SYSDATE AS REFRESHDATE,
lpad(Substr(Omf_Get_Alias(''FIN NBR'', vpp.Encntr_Id),1,50),12,''0'') encounter,
Lpad(Substr(Omf_Get_Alias(''MRN'', vpp.Encntr_Id),1,50),8,''0'') mrn,
(omf_get_pers_full(vpp.person_id)) patient_name, 
AGE,
UNIT,
MED_SERVICE,
ICU_START_DT_TM,
ICU_START_DT_TM+1 ICU_START_24_DT_TM
from vents_on_pop vpp
'


# **eFigure 2. SQL Code snippet for vital signs**

In [0]:
set @TSQL_icu_vals = 
'SELECT
    ce.encntr_id, 
   	CE.EVENT_END_DT_TM,
    event_cd, 
    PI_GET_cv_display(ce.event_cd) display, 
    substr(CE.RESULT_VAL, 1, 20) result,
    PI_GET_CV_DISPLAY(CE.RESULT_UNITS_CD) result_unit,
    to_date(''' + @vICUStartDT  + ''',''MON DD YYYY HH:MI AM'') as icu_start_dt_tm,
    to_date(''' + @vICUStart24DT  + ''',''MON DD YYYY HH:MI AM'') as icu_start_24_dt_tm,
	PI_GET_CV_DESC(CE.NORMALCY_CD) NORMALCY_IND,
	CE.VALID_FROM_DT_TM
from
 cust_utc.vw_clinical_event ce
WHERE ce.encntr_id = '+ @vencntr_id + '
 and  ce.event_cd in ('+ @listOfEvents + ')
 and  ce.valid_until_dt_tm > sysdate 
 and  CE.RESULT_STATUS_CD in (26,35,24) -- (AUTH,MODIFIED,MODIFIED2) 
 and  CE.EVENT_CLASS_CD <> 1266803  --Place Holder	
 and  CE.VIEW_LEVEL <> 0 
 AND  CE.RECORD_STATUS_CD = 186
 --AND  CE.EVENT_CLASS_CD + 0 = 229 --include only NUM values  COMMENTED OUT TO ALLOW Ox THerapy
 AND (
       (     ce.event_end_dt_tm >= to_date(''' + @vICUStartDT  + ''',''MON DD YYYY HH:MI AM'')
        AND  ce.event_end_dt_tm <= to_date(''' + @vICUStart24DT  + ''',''MON DD YYYY HH:MI AM'')
	   )
      OR
	   ( ce.event_end_dt_tm >= sysdate -2 and ce.event_end_dt_tm <= sysdate )
	 )	
' 


# **eFigure 3. SQL Code snippet for laboratory results**

In [0]:
SELECT @TSQL = 
			N'INSERT INTO #VENT_ICU_Labs_Stg
				(	encntr_id ,
					event_end_dt_tm,
					lab_type ,
					code_value,
					lab_result,
					lab_result_unit,
					normalcy_ind
				)
			SELECT * FROM OPENQUERY(ORACLE_CERNERP1592, 
			''SELECT 
				ce.encntr_id, 
				CCLSQL_CNVTDATETIMEUTC(CE.EVENT_END_DT_TM, 2, 145) event_end_dt_tm, 
				PI_GET_cv_display(ce.event_cd), 
				ce.event_cd,
				substr(CE.RESULT_VAL, 1, 20),
				PI_GET_CV_DISPLAY(CE.RESULT_UNITS_CD) "result_units_disp",
				PI_GET_CV_DESC(CE.NORMALCY_CD) NORMALCY_IND
      
				from  CLINICAL_EVENT CE

				WHERE 
				 ce.event_cd in (' + @listOfCodeValue + ')
				 and ce.encntr_id = ' + @e_enc + '
				 and ((
					ce.event_end_dt_tm >= CCLSQL_CNVTDATETIMEUTC(to_date(''''' + @icu_st_chr + ''''', ''''YYYY-MM-DD HH24:MI:SS'''') , 1, 145)
					and ce.event_end_dt_tm <= CCLSQL_CNVTDATETIMEUTC(to_date(''''' + @icu_24h_chr + ''''', ''''YYYY-MM-DD HH24:MI:SS'''') , 1, 145)
					) or
					(
					ce.event_end_dt_tm >= CCLSQL_CNVTDATETIMEUTC(to_date(''''' + @rptst_date + ''''', ''''YYYY-MM-DD HH24:MI:SS'''') , 1, 145)
					))
				 and CE.RESULT_STATUS_CD in (26,35,24) -- (AUTH,MODIFIED,MODIFIED2) 
				 and CE.EVENT_CLASS_CD <> 1266803  --Place Holder	
				 and CE.VIEW_LEVEL <> 0 
				 and CE.RECORD_STATUS_CD = 186
				 and ce.valid_until_dt_tm > CCLSQL_CNVTDATETIMEUTC(sysdate, 1, 145)
				 and not(regexp_like(CE.RESULT_VAL, ''''[a-z,A-Z]''''))
			''
			)';
